In [1]:
# Dependencies

import torch
import networkx as nx
from sklearn.base import BaseEstimator, ClassifierMixin
from datasets import load_dataset
from sklearn.model_selection import cross_val_score
from graph import process_dataset
import sys

sys.path.append("../")

import thdc
from hdc import pm


torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.empty_cache()
# torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [2]:
DATASET = load_dataset("graphs-datasets/MUTAG")["train"]
# DATASET = load_dataset("graphs-datasets/PROTEINS")["train"]

In [22]:
FOLDS, DIMENSIONS = 10, 2000
(graphs, labels) = process_dataset(DATASET)
VECTORS = torch.randint(0, 2, (100, DIMENSIONS), dtype=torch.float64).cuda()
VECTORS[VECTORS == 0] = -1

MAT = torch.from_numpy(pm(DIMENSIONS)).cuda()
MAT.type(torch.int8)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int8)

In [21]:
torch.matmul(VECTORS[0], MAT)

RuntimeError: expected scalar type Float but found Double

In [4]:
def encode(graph, vectors, mat, start):
    G = None
    for vs in nx.bfs_layers(graph, start):
        indices = torch.tensor(vs)
        if G is None:
            G = torch.sum(
                torch.index_select(vectors, 0, indices),
                dim=0,
            )
        else:
            G = torch.sum(
                torch.cat(
                    [
                        torch.matmul(G, mat)[None, :],
                        torch.index_select(vectors, 0, indices),
                    ],
                    0,
                ),
                dim=0,
            )
    return G

In [6]:
class GraphClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.memory = thdc.ItemMemory()

    def fit(self, X, y):
        for i in range(len(X)):
            for ii in range(1):
                self.memory.add_vector(str(y[i]), encode(X[i], VECTORS, MAT, start=ii))

        return self

    def predict(self, X):
        p = []
        for query in X:
            (label, _, _) = self.memory.cleanup(encode(query, VECTORS, MAT, 0))
            p.append(int(label))

        return p

In [7]:
def main():
    clf = GraphClassifier()
    scores = cross_val_score(
        clf,
        graphs,
        labels,
        cv=5,
        n_jobs=-1,
        verbose=4,
        error_score="raise",
    )
    print("Acc =>", scores.mean())


main()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


RuntimeError: expected scalar type Half but found Double